In [13]:
# boilerplate imports, trim later
import math
from random import sample
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from livelossplot import PlotLosses

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Dataset Loading

In [9]:
mafia_game_video = torchvision.io.VideoReader("datasets/game/MafiaVideogame.mp4", stream='video')

AttributeError: module 'torchvision' has no attribute 'io'